<a href="https://colab.research.google.com/github/MSSAMAJA/datascience_sql/blob/main/Datascience_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
# Set-up
%load_ext sql
%sql sqlite://
import pandas as pd

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [65]:
%%sql
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(255)
);

INSERT INTO Customers (CustomerID, CustomerName) VALUES
(1, 'Alice'),
(2, 'Bob'),
(3, 'Charlie'),
(4, 'David');

 * sqlite://
(sqlite3.OperationalError) table Customers already exists
[SQL: CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(255)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [66]:
%%sql
CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate DATE,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

INSERT INTO Orders (OrderID, CustomerID, OrderDate) VALUES
(101, 1, '2024-10-01'),
(102, 1, '2024-10-03'),
(103, 2, '2024-10-05');

 * sqlite://
(sqlite3.OperationalError) table Orders already exists
[SQL: CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate DATE,
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [67]:
#innerjoin
%%sql
select Customers.CustomerID,Orders.OrderID
from Customers
INNER JOIN Orders ON Customers.CustomerID==Orders.OrderID;


 * sqlite://
Done.


CustomerID,OrderID


In [68]:
#left join
%%sql
SELECT
    Customers.CustomerID,
    Customers.CustomerName,
    Orders.OrderID,
    Orders.OrderDate
FROM
    Customers
LEFT JOIN
    Orders ON Customers.CustomerID = Orders.CustomerID;

 * sqlite://
Done.


CustomerID,CustomerName,OrderID,OrderDate
1,Alice,101,2024-10-01
1,Alice,102,2024-10-03
2,Bob,103,2024-10-05
3,Charlie,None,None
4,David,None,None


In [69]:
%%sql
SELECT
    Customers.CustomerID,
    Customers.CustomerName,
    Orders.OrderID,
    Orders.OrderDate
FROM
    Customers
FULL JOIN
    Orders ON Customers.CustomerID = Orders.CustomerID;

 * sqlite://
(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: SELECT
    Customers.CustomerID,
    Customers.CustomerName,
    Orders.OrderID,
    Orders.OrderDate
FROM
    Customers
FULL JOIN
    Orders ON Customers.CustomerID = Orders.CustomerID;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [70]:
%%sql
CREATE TABLE Sales (
    SaleID INT PRIMARY KEY,
    Category VARCHAR(255),
    Amount DECIMAL(10, 2)
);

INSERT INTO Sales (SaleID, Category, Amount) VALUES
(1, 'Electronics', 1500.00),
(2, 'Clothing', 250.50),
(3, 'Electronics', 700.00),
(4, 'Clothing', 350.75),
(5, 'Home', 600.00),
(6, 'Electronics', 1250.00);


 * sqlite://
Done.
6 rows affected.


[]

In [71]:
#Write a query to find the average sales per category from a sales table.

In [72]:
%%sql
select SaleID,Category,Amount,AVG(Amount) AS avg_sales
from Sales
group by Category


 * sqlite://
Done.


SaleID,Category,Amount,avg_sales
2,Clothing,250.5,300.625
1,Electronics,1500,1150.0
5,Home,600,600.0


In [73]:
%%sql
DROP TABLE Sales;

 * sqlite://
Done.


[]

In [74]:
%%sql
CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    Name VARCHAR(100),
    Department VARCHAR(50),
    Salary DECIMAL(10, 2),
    HireDate DATE
);

INSERT INTO Employees (EmployeeID, Name, Department, Salary, HireDate) VALUES
(1, 'Alice', 'HR', 60000, '2021-05-10'),
(2, 'Bob', 'Engineering', 80000, '2019-11-20'),
(3, 'Charlie', 'HR', 65000, '2020-07-15'),
(4, 'David', 'Marketing', 55000, '2023-01-05'),
(5, 'Eva', 'Engineering', 90000, '2018-03-22'),
(6, 'Frank', 'Marketing', 75000, '2022-08-10');

 * sqlite://
(sqlite3.OperationalError) table Employees already exists
[SQL: CREATE TABLE Employees (
    EmployeeID INT PRIMARY KEY,
    Name VARCHAR(100),
    Department VARCHAR(50),
    Salary DECIMAL(10, 2),
    HireDate DATE
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [75]:
%%sql
CREATE TABLE Sales (
    SaleID INT PRIMARY KEY,
    EmployeeID INT,
    SaleAmount DECIMAL(10, 2),
    SaleDate DATE,
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID)
);

INSERT INTO Sales (SaleID, EmployeeID, SaleAmount, SaleDate) VALUES
(1, 1, 5000, '2023-09-12'),
(2, 2, 12000, '2023-09-15'),
(3, 3, 7000, '2023-09-20'),
(4, 5, 9500, '2023-09-22'),
(5, 4, 3500, '2023-10-01'),
(6, 6, 8000, '2023-10-05');

 * sqlite://
Done.
6 rows affected.


[]

In [76]:
#Write a query to find all employees who have a higher salary than the average salary of their department.
%%sql
select Name,Salary
from Employees e
WHERE Salary> (SELECT AVG(Salary) From Employees WHERE e.Department==Department) ;

 * sqlite://
Done.


Name,Salary
Charlie,65000
Eva,90000
Frank,75000


In [77]:
#Window Function (RANK):
#Write a query to rank employees by their salary within their department, with the highest salary getting rank 1
%%sql
select name,Salary,
      RANK() OVER(PARTITION BY Department ORDER BY Salary DESC)
from Employees


 * sqlite://
Done.


Name,Salary,RANK() OVER(PARTITION BY Department ORDER BY Salary DESC)
Eva,90000,1
Bob,80000,2
Charlie,65000,1
Alice,60000,2
Frank,75000,1
David,55000,2


In [78]:
#CTE (Common Table Expression):
#Write a CTE to calculate the total sales made by each employee, and then filter only those employees whose total sales exceed $10,000.
%%sql

with totalsales as(
  Select Employees.name,Employees.EmployeeID,sum(Sales.SaleAmount) as totalsalesamount
  from Employees
  join Sales on Employees.EmployeeID==Sales.EmployeeID
  group by Employees.name,Employees.EmployeeID)
select name,EmployeeID,totalsalesamount
from totalsales
where totalsalesamount>10000

 * sqlite://
Done.


name,EmployeeID,totalsalesamount
Bob,2,12000


In [88]:
#LEAD/LAG Function:
#Write a query using the LAG function to show each employee's salary and the difference between their salary and the previous employee’s salary (in descending order of salary).

%%sql
with salarytable as(
select Salary,name,LAG(Salary,1,0) OVER (ORDER BY Salary DESC) AS prev_sal
from Employees)
select name, Salary, prev_sal, Salary-prev_sal AS differenceinsal
from salarytable;


 * sqlite://
Done.


name,Salary,prev_sal,differenceinsal
Eva,90000,0,90000
Bob,80000,90000,-10000
Frank,75000,80000,-5000
Charlie,65000,75000,-10000
Alice,60000,65000,-5000
David,55000,60000,-5000


In [93]:
#Subquery with JOIN:
#Write a query to find the employee(s) who made the highest sale in the Sales table. Use a subquery to identify the highest sale amount and join it with the employees.
%%sql
select Employees.EmployeeID,Employees.name,Sales.SaleAmount
from Employees
Join Sales on Sales.EmployeeID==Employees.EmployeeID
WHERE Sales.SaleAmount==(SELECT max(Sales.SaleAmount) from Sales);

 * sqlite://
Done.


EmployeeID,Name,SaleAmount
2,Bob,12000


In [96]:
#Aggregate with HAVING:
#Write a query to display the departments where the average salary of employees is greater than $70,000.
%%sql
select Department,Salary,avg(Salary) as avg_sal
from Employees
group by Department
having avg_sal>70000;


 * sqlite://
Done.


Department,Salary,avg_sal
Engineering,80000,85000.0


In [99]:
#Window Function (ROW_NUMBER):
#Write a query using ROW_NUMBER() to list employees and their salaries, with a unique rank assigned to each employee. Reset the rank for each department and order by salary within each department.
%%sql
select name,
        Salary,
        ROW_NUMBER() OVER(PARTITION BY Department ORDER BY Salary) as row_number
from Employees

 * sqlite://
Done.


Name,Salary,row_number
Bob,80000,1
Eva,90000,2
Alice,60000,1
Charlie,65000,2
David,55000,1
Frank,75000,2


In [102]:
%%sql
select name,
        Salary,
        RANK() OVER( ORDER BY Salary) as row_number
from Employees

 * sqlite://
Done.


Name,Salary,row_number
David,55000,1
Alice,60000,2
Charlie,65000,3
Frank,75000,4
Bob,80000,5
Eva,90000,6


In [104]:
%%sql
select name,
        Salary,
        DENSE_RANK() OVER(ORDER BY Salary) as row_number
from Employees

 * sqlite://
Done.


Name,Salary,row_number
David,55000,1
Alice,60000,2
Charlie,65000,3
Frank,75000,4
Bob,80000,5
Eva,90000,6


In [108]:
#DATAMANIPULATION
#INSERT Statement:
#Write a query to insert a new employee record into the Employees table with the following details: EmployeeID = 8, Name = 'Henry', Department = 'IT', Salary = 75000, HireDate = '2023-08-01'.
%%sql
INSERT INTO Employees(EmployeeID,Name,Department,Salary,HireDate)
VALUES(8,'Henry','IT',75000,'2023-08-01')


 * sqlite://
1 rows affected.


[]

In [111]:
#UPDATE Statement:
#Write a query to increase the salary of all employees in the Marketing department by 15%.
%%sql
UPDATE Employees
SET Salary=Salary*0.15
WHERE Department='Marketing'


 * sqlite://
2 rows affected.


[]

In [113]:
#DELETE Statement:
#Write a query to delete all records from the Sales table where the SaleAmount is less than $5,000.
%%sql
DELETE FROM Sales
where SaleAmount<5000;

 * sqlite://
1 rows affected.


[]

In [117]:
#Transaction Control:
#Write a series of SQL commands to begin a transaction that updates the salary of all employees in the Finance department by 20% and then deletes all employees hired before January 1, 2019. Ensure that the transaction is committed only if both operations are successful.
#sqlite can't do transaction code is correct

%%sql
BEGIN TRANSACTION

UPDATE Employees
SET Salary=Salary*0.20
WHERE Department='Finance';

DELETE FROM Employees
WHERE HireDate<'2019-01-01';

commit;